In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.4)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.71 sec - Loss 0.167834 - ACC 70.53% - ACC Mean 70.53% - AUC 75.80% - AUC Mean 75.80% - Deter 000
Ite 00050 - 1.02 sec - Loss 0.067423 - ACC 66.96% - ACC Mean 68.33% - AUC 71.64% - AUC Mean 73.74% - Deter 020
Ite 00080 - 1.01 sec - Loss 0.050919 - ACC 67.86% - ACC Mean 68.15% - AUC 72.26% - AUC Mean 72.56% - Deter 050
Early stop ite 80, rollback to correction of ite 30, whith acc of 69.56% and auc of 76.36%
OUR METHOD RUN 1/10 - acc: 69.56% +- 0.0% - auc: 76.36% +- 0.0%
Ite 00000 - 1.22 sec - Loss 0.183274 - ACC 67.49% - ACC Mean 67.49% - AUC 72.49% - AUC Mean 72.49% - Deter 000
Ite 00050 - 0.94 sec - Loss 0.085060 - ACC 69.51% - ACC Mean 68.31% - AUC 73.26% - AUC Mean 73.29% - Deter 034
Ite 00066 - 0.96 sec - Loss 0.069689 - ACC 67.19% - ACC Mean 67.95% - AUC 72.81% - AUC Mean 72.62% - Deter 050
Early stop ite 66, rollback to correction of ite 16, whith acc of 70.68% and auc of 76.87%
OUR METHOD RUN 2/10 - acc: 70.68% +- 0.0% - auc: 76.87% +- 0.0%
Ite 00000 - 1.00 sec - L

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 71.29% +- 0.0% - auc: 74.67% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 51.67% +- 3.33% - auc: 59.13% +- 1.95%
SFIL RUN 2/10 - acc: 51.71% +- 2.67% - auc: 58.03% +- 3.34%
SFIL RUN 3/10 - acc: 51.76% +- 1.99% - auc: 57.09% +- 3.34%
SFIL RUN 4/10 - acc: 51.93% +- 3.1% - auc: 56.16% +- 1.45%
SFIL RUN 5/10 - acc: 52.41% +- 2.17% - auc: 58.47% +- 3.08%
SFIL RUN 6/10 - acc: 52.33% +- 2.22% - auc: 56.87% +- 3.14%
SFIL RUN 7/10 - acc: 51.62% +- 2.16% - auc: 58.44% +- 2.13%
SFIL RUN 8/10 - acc: 53.11% +- 2.64% - auc: 56.38% +- 1.69%
SFIL RUN 9/10 - acc: 51.72% +- 1.65% - auc: 57.45% +- 2.92%
SFIL RUN 10/10 - acc: 54.44% +- 2.36% - auc: 57.71% +- 1.81%
SFIL GLOBAL - acc: 52.27% +- 0.85% - auc: 57.57% +- 0.91%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 70.7% +- 3.79% - auc: 74.5% +- 3.85%
PFIL RUN 2/10 - acc: 68.3% +- 4.62% - auc: 73.84% +- 2.51%
PFIL RUN 3/10 - acc: 70.85% +- 3.38% - auc: 75.33% +- 3.93%
PFIL RUN 4/10 - acc: 70.69% +- 4.36% - auc: 75.82% +- 4.54%
PFIL RUN 5/10 - acc: 70.81% +- 3.41% - auc: 75.05% +- 2.28%
PFIL RUN 6/10 - acc: 71.26% +- 3.33% - auc: 73.04% +- 3.42%
PFIL RUN 7/10 - acc: 70.74% +- 3.48% - auc: 76.28% +- 1.11%
PFIL RUN 8/10 - acc: 70.96% +- 4.83% - auc: 75.44% +- 4.26%
PFIL RUN 9/10 - acc: 71.43% +- 3.14% - auc: 74.79% +- 2.94%
PFIL RUN 10/10 - acc: 70.27% +- 2.76% - auc: 75.02% +- 2.38%
PFIL GLOBAL - acc: 70.6% +- 0.82% - auc: 74.91% +- 0.89%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 54.19% +- 3.4% - auc: 57.42% +- 3.64%
SPOL RUN 2/10 - acc: 55.95% +- 3.12% - auc: 60.27% +- 2.8%
SPOL RUN 3/10 - acc: 59.89% +- 4.42% - auc: 63.02% +- 6.39%
SPOL RUN 4/10 - acc: 55.33% +- 4.55% - auc: 63.86% +- 7.01%
SPOL RUN 5/10 - acc: 57.29% +- 6.36% - auc: 60.71% +- 5.08%
SPOL RUN 6/10 - acc: 54.49% +- 1.83% - auc: 61.0% +- 3.57%
SPOL RUN 7/10 - acc: 57.89% +- 4.4% - auc: 60.0% +- 1.58%
SPOL RUN 8/10 - acc: 56.19% +- 3.84% - auc: 59.64% +- 5.49%
SPOL RUN 9/10 - acc: 54.53% +- 4.35% - auc: 57.76% +- 4.19%
SPOL RUN 10/10 - acc: 55.05% +- 3.49% - auc: 56.89% +- 3.99%
SPOL GLOBAL - acc: 56.08% +- 1.72% - auc: 60.06% +- 2.17%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 70.06% +- 4.65% - auc: 71.23% +- 3.94%
PPOL RUN 2/10 - acc: 69.44% +- 2.69% - auc: 73.47% +- 2.52%
PPOL RUN 3/10 - acc: 72.18% +- 3.76% - auc: 76.92% +- 2.76%
PPOL RUN 4/10 - acc: 71.45% +- 5.21% - auc: 74.46% +- 4.78%
PPOL RUN 5/10 - acc: 72.13% +- 2.96% - auc: 74.31% +- 1.98%
PPOL RUN 6/10 - acc: 70.4% +- 4.92% - auc: 73.09% +- 4.9%
PPOL RUN 7/10 - acc: 69.28% +- 4.54% - auc: 72.81% +- 2.98%
PPOL RUN 8/10 - acc: 71.09% +- 4.08% - auc: 75.66% +- 5.47%
PPOL RUN 9/10 - acc: 69.91% +- 3.7% - auc: 73.91% +- 2.31%
PPOL RUN 10/10 - acc: 71.07% +- 4.14% - auc: 74.62% +- 3.48%
PPOL GLOBAL - acc: 70.7% +- 0.99% - auc: 74.05% +- 1.49%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly worse than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly worse than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
